In [1]:
import pandas as pd

In [2]:
pair_sentence_info = pd.read_csv("../data/top_10_similar_data.csv")
pair_sentence_info.head()

,module,rule_id,material_id,source_1,source_2,chunk_1,chunk_2,score
0,与保障相关的时间,r_00025,m_00025a,INSURE_NOTICE,LIABILITY_EXCLUSION,(2)有效身份证件。 公司收到退保申请,份证件。自本公司收到您解除合同的通知书时,0.694824
1,与保障相关的时间,r_00025,m_00025a,INSURE_NOTICE,LIABILITY_EXCLUSION,(2)有效身份证件。 公司收到退保申请,超过10元的工本费后退还您所支付的全部保,0.656738
2,与保障相关的时间,r_00025,m_00025a,INSURE_NOTICE,LIABILITY_EXCLUSION,(2)有效身份证件。 公司收到退保申请,除合同通知书，并提供您的保险合同及有效身,0.606445
3,与保障相关的时间,r_00025,m_00025a,INSURE_NOTICE,LIABILITY_EXCLUSION,(2)有效身份证件。 公司收到退保申请,险费（本产品提供电子保单，犹豫期撤保免收,0.588379
4,与保障相关的时间,r_00025,m_00025a,INSURE_NOTICE,LIABILITY_EXCLUSION,(2)有效身份证件。 公司收到退保申请,以在此期间提出解除本合同，本公司将扣除不,0.567383


In [3]:
pair_sentence_info = pair_sentence_info[(pair_sentence_info['score'] > 0.8)]

In [4]:
pair_sentence_info

,module,rule_id,material_id,source_1,source_2,chunk_1,chunk_2,score
570,与保障相关的时间,r_00113,m_00017a,CLAUSE,INSURE_NOTICE,............\n犹豫期 ...,四、请您了解“犹豫期”的有关约定,0.807617
620,与保障相关的时间,r_00113,m_00017a,CLAUSE,INSURE_NOTICE,1.5.\n犹豫期 本合同生效后，自您收,合同生效后，自签收日次日起算犹豫期，投保,0.812012
650,与保障相关的时间,r_00113,m_00017a,CLAUSE,INSURE_NOTICE,不承担保险责任。我们将无息退还您已交的全,自始不承担保险责任。本公司将无息退还投保,0.851562
651,与保障相关的时间,r_00113,m_00017a,CLAUSE,INSURE_NOTICE,不承担保险责任。我们将无息退还您已交的全,我公司将无息退还您已交的全部保险费，犹豫,0.810547
850,与保障相关的时间,r_00113,m_00017a,CLAUSE,INSURE_NOTICE,您未按本合同约定日期交付续期保险费的，,付保费，不按时交费可能会影响您的权益。建,0.822266
...,...,...,...,...,...,...,...,...
524707,附加条款,r_00195,m_00137a,ADDITIONAL_AGREEMENT,INSURE_NOTICE,额-免赔额抵扣部分）]*100%。 免赔,（免赔额-免赔额抵扣部分）方案名称保险责,0.841309
524708,附加条款,r_00195,m_00137a,ADDITIONAL_AGREEMENT,INSURE_NOTICE,额-免赔额抵扣部分）]*100%。 免赔,于抵扣免赔额，但从其他途径已获得的医疗费,0.834961
524709,附加条款,r_00195,m_00137a,ADDITIONAL_AGREEMENT,INSURE_NOTICE,额-免赔额抵扣部分）]*100%。 免赔,）]*100%免赔额抵扣部分为其它第三方,0.827148
524716,附加条款,r_00195,m_00137a,ADDITIONAL_AGREEMENT,INSURE_NOTICE,额抵扣部分为其它第三方获得的医疗费用补偿,额-其它第三方获得的医疗费用补偿金额）-,0.805176


In [5]:
import vllm
import numpy as np
import pandas as pd
from transformers import PreTrainedTokenizer, AutoTokenizer
from typing import List
import torch
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
import re

model_path = "../Qwen3-14B-AWQ/"
tokenizer = AutoTokenizer.from_pretrained(model_path)


def preprocess_text(x):
    x = re.sub("http\w+", '',x)   # Delete URL
    x = re.sub(r"\.+", ".", x)    # Replace consecutive commas and periods with one comma and period character
    x = re.sub(r"\,+", ",", x)
    x = re.sub(r"\\\(", " ", x)
    x = re.sub(r"\\\)", " ", x)
    x = re.sub(r"[ ]{1,}", " ", x)
    x = x.strip()                 # Remove empty characters at the beginning and end
    return x

llm = vllm.LLM(
    model_path,
    quantization="awq",
    tensor_parallel_size=2,
    gpu_memory_utilization=0.95, 
    trust_remote_code=True,
    dtype="half", 
    enforce_eager=True,
    max_model_len=8192,
    disable_log_stats=True
)
tokenizer = llm.get_tokenizer()

<>:15: SyntaxWarning: invalid escape sequence '\w'
<>:15: SyntaxWarning: invalid escape sequence '\w'
/tmp/ipykernel_2628/3548283683.py:15: SyntaxWarning: invalid escape sequence '\w'
  x = re.sub("http\w+", '',x)   # Delete URL


INFO 06-28 08:42:13 [__init__.py:243] Automatically detected platform cuda.
INFO 06-28 08:42:16 [__init__.py:31] Available plugins for group vllm.general_plugins:
INFO 06-28 08:42:16 [__init__.py:33] - lora_filesystem_resolver -> vllm.plugins.lora_resolvers.filesystem_resolver:register_filesystem_resolver
INFO 06-28 08:42:16 [__init__.py:36] All plugins in this group will be loaded. Set `VLLM_PLUGINS` to control which plugins to load.
INFO 06-28 08:42:26 [config.py:793] This model supports multiple tasks: {'generate', 'embed', 'classify', 'score', 'reward'}. Defaulting to 'generate'.
INFO 06-28 08:42:27 [awq_marlin.py:119] Detected that the model can run with awq_marlin, however you specified quantization=awq explicitly, so forcing awq. Use quantization=awq_marlin for faster inference
WARNING 06-28 08:42:27 [config.py:907] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 06-28 08:42:27 [config.py:1875] Defaulting to use mp for distrib

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


(VllmWorker rank=1 pid=2799) INFO 06-28 08:42:32 [default_loader.py:280] Loading weights took 2.53 seconds
(VllmWorker rank=0 pid=2796) INFO 06-28 08:42:32 [default_loader.py:280] Loading weights took 2.63 seconds
(VllmWorker rank=1 pid=2799) INFO 06-28 08:42:32 [gpu_model_runner.py:1549] Model loading took 4.6923 GiB and 2.789620 seconds
(VllmWorker rank=0 pid=2796) INFO 06-28 08:42:33 [gpu_model_runner.py:1549] Model loading took 4.6923 GiB and 2.883033 seconds
INFO 06-28 08:42:36 [kv_cache_utils.py:637] GPU KV cache size: 203,456 tokens
INFO 06-28 08:42:36 [kv_cache_utils.py:640] Maximum concurrency for 8,192 tokens per request: 24.84x
INFO 06-28 08:42:36 [kv_cache_utils.py:637] GPU KV cache size: 203,456 tokens
INFO 06-28 08:42:36 [kv_cache_utils.py:640] Maximum concurrency for 8,192 tokens per request: 24.84x
INFO 06-28 08:42:36 [core.py:167] init engine (profile, create kv cache, warmup model) took 3.25 seconds


In [6]:
PROMPT  = """你是一个专业的保险行业的信息处理专家，请对给定两段保险相关文本进行分析，判断两个文本所涉及到的{module}定义是否有冲突(即相同条件的表达存在不一致)，只需要回答是或否。
            
{module}解释: {expanation}

文本一: {content1}

文本二: {content2}

注意：若两个文本描述的东西不相似，请直接返回否

"""

In [7]:
def apply_template(row, tokenizer):
    expanation = {
        '基础产品销售信息': '该保险产品的基础配置信息，包括产品名、附加的条款信息、销售限制等',
        '投保条款': '约定该产品的保险责任细节，如保障范围、保险金额、增值服务等',
        '与保障相关的时间': '约定该产品的各类时间信息，包括但不限于犹豫期、等待期、宽限期等',
        '保障相关时间': '约定该产品的各类时间信息，包括但不限于犹豫期、等待期、宽限期等',
        "保障责任": "约定该产品的保险责任细节，如保障范围、保险金额、增值服务等；",
        '赔付 & 领取规则': '约定该产品的保险责任的赔付、给付、领取及免赔细节，如赔付年龄/比例/次数等',
        '责任免除': '约定该产品不承担保险责任的情形',
        '续保条款': '约定续保相关信息，包括但不限于续保条件、保证续保等',
        '退保条款': '约定退保相关信息，包括但不限于退保条件、退保手续费等',
        '出险条款': '约定出险相关信息，包括但不限于出险地点、出险方式等',
        '附加条款': '约定该产品的附加条款，如特别约定等',
        '术语解释': '约定该产品的术语解释，如名词定义等'
    }
    messages = [
        {"role": "user", 
         "content": preprocess_text(
                PROMPT.format(
                    expanation=expanation[row['module']],
                    module=row['module'],
                    content1=row['chunk_1'],
                    content2=row['chunk_2'],
                )
            )},
    ]
    text = tokenizer.apply_chat_template(messages, 
                                         tokenize=False, 
                                         add_generation_prompt=True,
                                        enable_thinking=False)
    return text

In [8]:
pair_sentence_info["text"] = pair_sentence_info.apply(lambda row: apply_template(row, tokenizer), axis=1)

print("Example:")
print(pair_sentence_info["text"].values[0])
print()


mclp = MultipleChoiceLogitsProcessor(tokenizer, choices=["是", "否"])

responses = llm.generate(
    pair_sentence_info["text"].values,
    vllm.SamplingParams(
        n=1,  # Number of output sequences to return for each prompt.
        top_k=1,  # Float that controls the cumulative probability of the top tokens to consider.
        temperature=0,  # randomness of the sampling
        seed=777, # Seed for reprodicibility
        skip_special_tokens=False,  # Whether to skip special tokens in the output.
        max_tokens=1,  # Maximum number of tokens to generate per output sequence.
        # logits_processors=[mclp],
        # logprobs=len(mclp.choices)
    ),
    use_tqdm=True,
    
)

# responses = [1 if x.outputs[0].text == 'Yes' else 0 for x in responses]
# print(responses)

Example:
<|im_start|>user
你是一个专业的保险行业的信息处理专家，请对给定两段保险相关文本进行分析，判断两个文本所涉及到的与保障相关的时间定义是否有冲突(即相同条件的表达存在不一致)，只需要回答是或否。
 
与保障相关的时间解释: 约定该产品的各类时间信息，包括但不限于犹豫期、等待期、宽限期等

文本一: .
犹豫期 .

文本二: 四、请您了解“犹豫期”的有关约定

注意：若两个文本描述的东西不相似，请直接返回否<|im_end|>
<|im_start|>assistant
<think>

</think>





Adding requests:   0%|          | 0/6965 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/6965 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

In [9]:
len(responses)

6965

In [10]:
answers = [1 if res.outputs[0].text == '是' else 0 for res in responses]

In [73]:
# logprobs = []
# for output in responses:
#     try:
#         lg = output.outputs[0].logprobs[0].values()
#         logprobs.append({lp.decoded_token: lp.logprob for lp in list(lps)})
#     except:
#         logprobs.append({"否": 1, "是": 0})

In [74]:
# # logprobs = []
# # for lps in [output.outputs[0].logprobs[0].values() for output in responses]:
# #     logprobs.append({lp.decoded_token: lp.logprob for lp in list(lps)})

# logit_matrix = pd.DataFrame(logprobs)[["是", "否"]].values

# choices = ["是", "否"]
# answers = [choices[pick] for pick in np.argmax(logit_matrix, axis=1)]

In [11]:
set(answers)

{0, 1}

In [12]:
pair_sentence_info['res'] = answers
pair_sentence_info

,module,rule_id,material_id,source_1,source_2,chunk_1,chunk_2,score,text,res
570,与保障相关的时间,r_00113,m_00017a,CLAUSE,INSURE_NOTICE,............\n犹豫期 ...,四、请您了解“犹豫期”的有关约定,0.807617,<|im_start|>user\n你是一个专业的保险行业的信息处理专家，请对给定两段保险相...,1
620,与保障相关的时间,r_00113,m_00017a,CLAUSE,INSURE_NOTICE,1.5.\n犹豫期 本合同生效后，自您收,合同生效后，自签收日次日起算犹豫期，投保,0.812012,<|im_start|>user\n你是一个专业的保险行业的信息处理专家，请对给定两段保险相...,1
650,与保障相关的时间,r_00113,m_00017a,CLAUSE,INSURE_NOTICE,不承担保险责任。我们将无息退还您已交的全,自始不承担保险责任。本公司将无息退还投保,0.851562,<|im_start|>user\n你是一个专业的保险行业的信息处理专家，请对给定两段保险相...,0
651,与保障相关的时间,r_00113,m_00017a,CLAUSE,INSURE_NOTICE,不承担保险责任。我们将无息退还您已交的全,我公司将无息退还您已交的全部保险费，犹豫,0.810547,<|im_start|>user\n你是一个专业的保险行业的信息处理专家，请对给定两段保险相...,1
850,与保障相关的时间,r_00113,m_00017a,CLAUSE,INSURE_NOTICE,您未按本合同约定日期交付续期保险费的，,付保费，不按时交费可能会影响您的权益。建,0.822266,<|im_start|>user\n你是一个专业的保险行业的信息处理专家，请对给定两段保险相...,1
...,...,...,...,...,...,...,...,...,...,...
524707,附加条款,r_00195,m_00137a,ADDITIONAL_AGREEMENT,INSURE_NOTICE,额-免赔额抵扣部分）]*100%。 免赔,（免赔额-免赔额抵扣部分）方案名称保险责,0.841309,<|im_start|>user\n你是一个专业的保险行业的信息处理专家，请对给定两段保险相...,1
524708,附加条款,r_00195,m_00137a,ADDITIONAL_AGREEMENT,INSURE_NOTICE,额-免赔额抵扣部分）]*100%。 免赔,于抵扣免赔额，但从其他途径已获得的医疗费,0.834961,<|im_start|>user\n你是一个专业的保险行业的信息处理专家，请对给定两段保险相...,1
524709,附加条款,r_00195,m_00137a,ADDITIONAL_AGREEMENT,INSURE_NOTICE,额-免赔额抵扣部分）]*100%。 免赔,）]*100%免赔额抵扣部分为其它第三方,0.827148,<|im_start|>user\n你是一个专业的保险行业的信息处理专家，请对给定两段保险相...,1
524716,附加条款,r_00195,m_00137a,ADDITIONAL_AGREEMENT,INSURE_NOTICE,额抵扣部分为其它第三方获得的医疗费用补偿,额-其它第三方获得的医疗费用补偿金额）-,0.805176,<|im_start|>user\n你是一个专业的保险行业的信息处理专家，请对给定两段保险相...,1


In [26]:
pair_sentence_info.to_csv('./define_classification_results.csv', index=False)

In [27]:
df_mean = pair_sentence_info.groupby(['material_id', 'rule_id'])['res'].mean().reset_index()
df_mean

,material_id,rule_id,res
0,m_00001a,r_00001,1.000000
1,m_00003a,r_00003,0.000000
2,m_00004a,r_00148,0.848341
3,m_00004a,r_00182,1.000000
4,m_00005a,r_00174,0.806452
...,...,...,...
77,m_00133a,r_00190,0.734694
78,m_00135a,r_00193,0.954545
79,m_00136a,r_00194,1.000000
80,m_00137a,r_00195,0.894737


In [28]:
df_mean.sort_values('res').head(10)

,material_id,rule_id,res
70,m_00124a,r_00175,0.000000
23,m_00032a,r_00036,0.000000
19,m_00022a,r_00022,0.000000
21,m_00027a,r_00029,0.000000
37,m_00061a,r_00074,0.000000
1,m_00003a,r_00003,0.000000
26,m_00036a,r_00092,0.000000
73,m_00129a,r_00180,0.000000
75,m_00131a,r_00186,0.250000
8,m_00009a,r_00009,0.299065


In [29]:
df_mean[df_mean['res'] == 0]

,material_id,rule_id,res
1,m_00003a,r_00003,0.0
19,m_00022a,r_00022,0.0
21,m_00027a,r_00029,0.0
23,m_00032a,r_00036,0.0
26,m_00036a,r_00092,0.0
37,m_00061a,r_00074,0.0
70,m_00124a,r_00175,0.0
73,m_00129a,r_00180,0.0


In [30]:
first_submit = pd.read_json("../data/submit_74.jsonl", lines=True)
first_submit

,material_id,rule_id,result
0,m_00001a,r_00001,False
1,m_00002a,r_00002,False
2,m_00003a,r_00003,False
3,m_00004a,r_00004,True
4,m_00005a,r_00005,False
...,...,...,...
195,m_00138a,r_00196,False
196,m_00120a,r_00197,False
197,m_00139a,r_00198,False
198,m_00061a,r_00199,False


In [31]:
first_submit[first_submit['result'] == True]

,material_id,rule_id,result
3,m_00004a,r_00004,True
8,m_00009a,r_00009,True
9,m_00010a,r_00010,True
10,m_00011a,r_00011,True
11,m_00012a,r_00012,True
...,...,...,...
183,m_00041a,r_00184,True
184,m_00088a,r_00185,True
185,m_00131a,r_00186,True
190,m_00134a,r_00191,True


In [32]:
# 合并两个表
df_updated = first_submit.merge(df_mean, on=['material_id', 'rule_id'], how='left')
df_updated

,material_id,rule_id,result,res
0,m_00001a,r_00001,False,1.000000
1,m_00002a,r_00002,False,NaN
2,m_00003a,r_00003,False,0.000000
3,m_00004a,r_00004,True,NaN
4,m_00005a,r_00005,False,NaN
...,...,...,...,...
195,m_00138a,r_00196,False,0.857143
196,m_00120a,r_00197,False,NaN
197,m_00139a,r_00198,False,NaN
198,m_00061a,r_00199,False,0.790909


In [33]:
df_updated['result'] = df_updated.apply(lambda x: True if x.res == 0 else x.result,axis=1)

In [34]:
df_updated['result'] = df_updated.apply(lambda x: False if x.res >= 0.8 else x.result,axis=1)

In [35]:
df_updated[df_updated['result'] == True]

,material_id,rule_id,result,res
2,m_00003a,r_00003,True,0.000000
3,m_00004a,r_00004,True,NaN
8,m_00009a,r_00009,True,0.299065
9,m_00010a,r_00010,True,NaN
10,m_00011a,r_00011,True,NaN
...,...,...,...,...
180,m_00032a,r_00181,True,NaN
183,m_00041a,r_00184,True,NaN
184,m_00088a,r_00185,True,0.500000
185,m_00131a,r_00186,True,0.250000


In [23]:
df_updated[['material_id', 'rule_id', 'result']].to_json('submit.jsonl', orient='records', lines=True)

In [26]:
# from markitdown import MarkItDown
# import time 

# start_time = time.time()

# md = MarkItDown()
# result = md.convert("../data/10.1002_2017jc013030.pdf")
# print(result.text_content)

# print(time.time() - start_time)